<a href="https://colab.research.google.com/github/AravindBiswas/MyStudy/blob/master/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://www.kaggle.com/crowdflower/twitter-airline-sentiment/home

--2025-04-27 06:05:34--  https://www.kaggle.com/crowdflower/twitter-airline-sentiment/home
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /datasets/crowdflower/twitter-airline-sentiment/home [following]
--2025-04-27 06:05:35--  https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/home
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘home.1’

home.1                  [ <=>                ]  10.16K  --.-KB/s    in 0.008s  

2025-04-27 06:05:35 (1.22 MB/s) - ‘home.1’ saved [10408]



In [ ]:
!pwd


/content


In [ ]:
#!find / -name "*.csv"
!ls /content/home

/content/home


In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Read the training data
def read_data(file_name):
    data = pd.read_csv(file_name)
    data = data[['airline_sentiment', 'text']]
    return data

import pandas as pd

# Assuming the file name is Tweets.csv (check the actual file name using !ls)
file_path = '/content/Tweets.csv'  # Update the file path based on the output of !pwd and !ls

# Read the CSV file
data = pd.read_csv(file_path)

# Print the first few lines of the file
print(data.head())

In [ ]:
# Observe randomly generated 10 tweets for each sentiment
def observe_tweets(data):
    for sentiment in data['airline_sentiment'].unique():
        print(f"Sentiment: {sentiment}")
        print(data[data['airline_sentiment'] == sentiment].sample(10))

In [ ]:
# Clean the tweet text
def clean_text(data):
    data['clean_text'] = data['text'].apply(lambda x: re.sub(r'[@#]\w*', '', x))
    data['clean_text'] = data['clean_text'].apply(lambda x: re.sub(r'http\S+', '', x))
    data['clean_text'] = data['clean_text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    return data

In [ ]:
# List down the most common 15 words for each sentiment
def common_words(data):
    for sentiment in data['airline_sentiment'].unique():
        print(f"Sentiment: {sentiment}")
        print(data[data['airline_sentiment'] == sentiment]['clean_text'].str.split().explode().value_counts().head(15))


In [ ]:
# Remove Stopwords
def remove_stopwords(data):
    stop_words = set(stopwords.words('english'))
    data['clean_text_no_stopwords'] = data['clean_text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))
    return data

In [ ]:
# Remove specific words
def remove_specific_words(data):
    words_to_remove = ['americanair', 'united', 'delta', 'southwestair', 'jetblue', 'virginamerica', 'usairways', 'flight', 'plane']
    data['clean_text_final'] = data['clean_text_no_stopwords'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in words_to_remove]))
    return data

In [ ]:
# Encode Sentiments
def encode_sentiments(data):
    le = LabelEncoder()
    data['sentiment_encoded'] = le.fit_transform(data['airline_sentiment'])
    return data

In [ ]:
# Vectorize the Text Column
def vectorize_text(data):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(data['clean_text_final'])
    return X

In [ ]:
# Prepare a multiclass Classification model
def prepare_model(X, y):
    model = MultinomialNB()
    model.fit(X, y)
    return model

In [ ]:
# Main function
def main():
    train_data = read_data('Tweets-train.csv')
    observe_tweets(train_data)
    train_data = clean_text(train_data)
    common_words(train_data)
    train_data = remove_stopwords(train_data)
    common_words(train_data)
    train_data = remove_specific_words(train_data)
    common_words(train_data)
    train_data = encode_sentiments(train_data)
    X = vectorize_text(train_data)
    y = train_data['sentiment_encoded']
    model = prepare_model(X, y)

    test_data = read_data('Tweets-test.csv')
    test_data = clean_text(test_data)
    test_data = remove_stopwords(test_data)
    test_data = remove_specific_words(test_data)
    test_data = encode_sentiments(test_data)
    X_test = vectorizer.transform(test_data['clean_text_final'])
    y_pred = model.predict(X_test)
    print("Confusion Matrix:")
    print(confusion_matrix(test_data['sentiment_encoded'], y_pred))
    print("Accuracy:", accuracy_score(test_data['sentiment_encoded'], y_pred))
    print("Classification Report:")
    print(classification_report(test_data['sentiment_encoded'], y_pred))

